In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
dataset = pd.read_csv('../../datasets/grocery_dataset/Grocery_Inventory.csv')

In [3]:
df = dataset[['Catagory', 'Date_Received', 'Last_Order_Date', 'Stock_Quantity', 'Unit_Price', 'Sales_Volume']]

In [4]:
df = df.fillna(method='ffill')

df['Unit_Price'] = df['Unit_Price'].replace(r'[\$,]', '', regex=True).astype(float)

df['Date_Received'] = pd.to_datetime(df['Date_Received'])
df['Last_Order_Date'] = pd.to_datetime(df['Last_Order_Date'])

C:\Users\Mahimna\AppData\Local\Temp\ipykernel_6540\3696563017.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [5]:
df['received_year'] = df['Date_Received'].dt.year
df['received_month'] = df['Date_Received'].dt.month
df['received_day'] = df['Date_Received'].dt.day

df['order_year'] = df['Last_Order_Date'].dt.year
df['order_month'] = df['Last_Order_Date'].dt.month
df['order_day'] = df['Last_Order_Date'].dt.day

In [6]:
df['days_between'] = (df['Last_Order_Date'] - df['Date_Received']).dt.days

In [7]:
for lag in [1, 3, 7]:
    df[f'sales_lag_{lag}'] = df['Sales_Volume'].shift(lag).fillna(method='bfill')

C:\Users\Mahimna\AppData\Local\Temp\ipykernel_6540\2596367736.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[f'sales_lag_{lag}'] = df['Sales_Volume'].shift(lag).fillna(method='bfill')
C:\Users\Mahimna\AppData\Local\Temp\ipykernel_6540\2596367736.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[f'sales_lag_{lag}'] = df['Sales_Volume'].shift(lag).fillna(method='bfill')
C:\Users\Mahimna\AppData\Local\Temp\ipykernel_6540\2596367736.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[f'sales_lag_{lag}'] = df['Sales_Volume'].shift(lag).fillna(method='bfill')


In [8]:
df['sales_roll_3'] = df['Sales_Volume'].rolling(window=3, min_periods=1).mean()
df['sales_roll_7'] = df['Sales_Volume'].rolling(window=7, min_periods=1).mean()

In [9]:
df['price_stock'] = df['Unit_Price'] * df['Stock_Quantity']

In [10]:
df = df.drop(columns=['Date_Received', 'Last_Order_Date'])

In [11]:
df = pd.get_dummies(df, columns=['Catagory'], drop_first=True)

In [12]:
X = df.drop(columns=['Sales_Volume'])
y = df['Sales_Volume']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [14]:
# model = LinearRegression()
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [15]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Linear Regression with lag features RMSE:", rmse)
print("R²:", r2)

Linear Regression with lag features RMSE: 16.227183374204017
R²: 0.48136490087674266


In [16]:
import joblib

# Save model
joblib.dump(model, "linear_model.pkl")

# Save feature names
joblib.dump(X.columns, "feature_columns.pkl")


['feature_columns.pkl']